### Saving and Loading Models
Here i will show you how to save and load the models with PyTorch. This is very important since we will load and unload previously used trained models to use in making predictions or to continue training on new data.

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt

import torch
from torch import nn, optim
import torch.nn.functional as F
from torch.autograd import Variable
from torchvision import datasets, transforms

import fc_model
import helper

In [ ]:

# Define a transform to normalize the data
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.5,), (0.5,))])
# Download and load the training data
trainset = datasets.FashionMNIST('F_MNIST_data/', download=True, train=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

# Download and load the test data
testset = datasets.FashionMNIST('F_MNIST_data/', download=True, train=False, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=True)

In [ ]:
images, labels = next(iter(trainloader))
helper.imshow(image[0,:1])

### Train a Network

In [ ]:
model = fc_model.Network(784, 10, [512, 256,128])
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

## Training the model
fc_model.train(model, trainloader, testloader, criterion, optimizer, epochs =2)


### Saving and loading the networks
It becomes quite easy if we save the network model and run the train model on new dataset to use them for predicitons

The parameters for PyTorch networks are stored in a model's state_dict. we can see that the state dict contains the weight abd bias matrices for each of our layers.

In [ ]:
print("Our model :\n\n", model, '\n')
print("The state_dict keys: \n\n", model.state_dict().keys)

### save the model

In [ ]:
##Simplest way to save the state dict with torch.save. for example, we can save 
# it to a file 'checkpoint.pth'
torch.save(model.state_dict(), 'checkpoint.pth')

### load the model

In [ ]:
state_dict = torch.load('checkpoint.pth')
print(state_dict.keys())

### load the state_dict in to the network

In [ ]:
model.load_state_dict(state_dict)

Straightforward, but as usual its a bit more complicated. Loading the state dict works only if the model architecture is exactly the same as the checkpoint architecture. if i create a model with a different architecure, this fails for example

In [ ]:

# Try this
model = fc_model.Network(784, 10, [400, 200, 100])
# This will throw an error because the tensor sizes are wrong!
model.load_state_dict(state_dict)

This means we need to rebuild the model exactly as it was when trained. Information about the model architecture also needs to be saved in the checkpoint, along with the state dict. 

In [ ]:
checkpoint = {'input_size':784,
              'output_size':10,
              'hidden_layers':[each.out_features for each in model.hidden_layers],
              'state_dict':model.state_dict()
             }
torch.save(checkpoint, 'checkpoint.pth')

Now the checkpoint has all the necessary information to rebuild the trained model. You can easily make that a function if you want. Similarly we can write a functions to load checkpoints

In [ ]:

def load_checkpoint(filepath):
    checkpoint = torch.load(filepath)
    model = fc_model.Network(checkpoint['input_size'],
                             checkpoint['output_size'],
                             checkpoint['hidden_layers'])
    model.load_state_dict(checkpoint['state_dict'])
    
    return model

model = load_checkpoint('checkpoint.pth')
print(model)